In [ ]:
pip install transformers torch transformers_stream_generator tiktoken einops accelerate

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers.generation import GenerationConfig
import warnings
warnings.filterwarnings("ignore")
import textwrap
import torch

### MODEL: tiiuae/falcon-7b-instruct

In [ ]:
model_path = "tiiuae/falcon-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path, torch_dtype=torch.float16, device_map='auto',trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

configuration_falcon.py:   0%|          | 0.00/7.16k [00:00<?, ?B/s]

...

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [ ]:
# Sample customer email text
text = """
Dear WidgetCorp,
I am writing to express my deep disappointment with your service regarding my recent order (Order #98765). I had ordered the "SuperWidget" as a birthday gift for my daughter,
Emily, whose birthday was on March 15th. Despite your website promising delivery within three days, the product arrived only yesterday, missing her birthday entirely.
I attempted to contact your customer service multiple times for an explanation and to expedite the delivery, but to no avail. My calls and emails went unanswered, leaving me
in a difficult position with no gift for Emily’s special day.This experience has left a sour taste, and I am reconsidering future purchases from WidgetCorp. I expect a prompt
explanation and an assurance that such delays will not occur in the future.
Sincerely,
Jane Doe
"""

qa_questions = [
    "Q1: What was the reason for the customer's disappointment with WidgetCorp?",
    "Q2: How did WidgetCorp's customer service respond to the customer's inquiries about the late delivery?"
]

prompt = f"""
Perform the following actions:
1 - Summarize the following text delimited by triple backticks.
2 - Identify the entities in the text delimited by triple backticks.
3 - Answer the following questions delimited by triple backticks based on the text.
4 - Translate the summary into French and back to English.

Separate your answers with line breaks.

Text:
```{text}```
Questions:
```{qa_questions}```
"""

### SUMMARIZATION TASK

In [ ]:
query = "Summarize the following text:"
prompt = query + "\n" + text
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
generation_output = model.generate(
    input_ids=input_ids, max_new_tokens=600)
print(tokenizer.decode(generation_output[0]))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1408: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
2024-01-30 16:34:05.018312: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-30 16:34:05.018439: E external/local_xla/xla/s

summarize the following text in your own words: 
Dear WidgetCorp,
I am writing to express my deep disappointment with your service regarding my recent order (Order
#98765). I had ordered the "SuperWidget" as a birthday gift for my daughter, Emily, whose birthday
was on March 15th. Despite your website promising delivery within three days, the product arrived
only yesterday, missing her birthday entirely.
I attempted to contact your customer service multiple times for an explanation and to expedite the
delivery, but to no avail. My calls and emails went unanswered, leaving me in a difficult position with
no gift for Emily’s special day.
This experience has left a sour taste, and I am reconsidering future purchases from WidgetCorp. I
expect a prompt explanation and an assurance that such delays will not occur in the future.
Sincerely,
Jane Doe
In summary, Jane Doe is disappointed with WidgetCorp's delayed delivery of her daughter's birthday gift, SuperWidget, and is considering future pu

### QUESTION ANSWERING TASK

In [ ]:
prompt = f"""Answer both questions:```{qa_questions}```"""
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
generation_output = model.generate(
    input_ids=input_ids, max_new_tokens=300)
print(tokenizer.decode(generation_output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1408: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Answer both questions:
            Q1: What was the reason for the customer's disappointment with WidgetCorp?
            Q2:How did WidgetCorp's customer service respond to the customer's inquiries about the late
A: The customer was dissatisfied with WidgetCorp because their product was not delivered on time.
B: WidgetCorp's customer service responded by offering a refund or compensation for the delay in delivery.<|endoftext|>


### NAME ENTITY RECOGNITION TASK

In [ ]:
query = "Identify the entities and their types using standard NER mechanism in the following text: "
prompt = query + "\n" + text
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
generation_output = model.generate(
    input_ids=input_ids, max_new_tokens=800)
print(tokenizer.decode(generation_output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


identify the entities and their types (use standard NER terminology) in the following text: 
Dear WidgetCorp,
I am writing to express my deep disappointment with your service regarding my recent order (Order
#98765). I had ordered the "SuperWidget" as a birthday gift for my daughter, Emily, whose birthday
was on March 15th. Despite your website promising delivery within three days, the product arrived
only yesterday, missing her birthday entirely.
I attempted to contact your customer service multiple times for an explanation and to expedite the
delivery, but to no avail. My calls and emails went unanswered, leaving me in a difficult position with
no gift for Emily’s special day.
This experience has left a sour taste, and I am reconsidering future purchases from WidgetCorp. I
expect a prompt explanation and an assurance that such delays will not occur in the future.
Sincerely,
Jane Doe
The entities in the text are:
- WidgetCorp
- Jane Doe
- Order #98765
- SuperWidget
- Emily
- WidgetCor

### TRANSLATION (English to French to English)

In [ ]:
query = "Translate the text two times: First time from English to French, second time translate the French text to English. Show both translations"
prompt = query + "\n" + "\n" + text + "\n" + "\n"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
generation_output = model.generate(
    input_ids=input_ids, max_new_tokens=800)
print(tokenizer.decode(generation_output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Translate the text two times: First time from English to French, second time translate the French text to English. Show both translations 

Dear WidgetCorp,
I am writing to express my deep disappointment with your service regarding my recent order (Order
#98765). I had ordered the "SuperWidget" as a birthday gift for my daughter, Emily, whose birthday
was on March 15th. Despite your website promising delivery within three days, the product arrived
only yesterday, missing her birthday entirely.
I attempted to contact your customer service multiple times for an explanation and to expedite the
delivery, but to no avail. My calls and emails went unanswered, leaving me in a difficult position with
no gift for Emily’s special day.
This experience has left a sour taste, and I am reconsidering future purchases from WidgetCorp. I
expect a prompt explanation and an assurance that such delays will not occur in the future.
Sincerely,
Jane Doe

(French translation)
Cher WidgetCorp,
Je suis écœuré d